In [7]:
!uv pip install -qU sentence-transformers
!uv pip install -q transformers
!uv pip install -q datasets 
!uv pip install -q ipywidgets

#### Research

In the provided notebook, research will be conducted on the performance and methods for fine-tuning a local, open-source embedding model to achieve better performance in the Ukrainian language.

#### Multilingual Distillation

We will start by researching the approach proposed in the following [research paper](https://arxiv.org/pdf/2004.09813).